# Vector AI coding Challenge
## Alberto Raimondi

In [1]:
# !pip3 install fuzzywuzzy
# !pip3 install python-Levenshtein
# !pip3 install cleanco

In [2]:
#insatlling spacy model for entity recognition
# !python -m spacy download en_core_web_sm

In [3]:
import usaddress
import string

from collections import Counter, defaultdict, OrderedDict
import pycountry
from fuzzywuzzy import fuzz, process
from cleanco import prepare_terms, basename

import spacy

nlp = spacy.load("en_core_web_sm")
company_terms = prepare_terms()

/home/alberto/misc/anaconda3/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


## First Task

The first task consists of creating a set of functions able to identify duplicate entities for specific kinds of inputs.
The data given is very small so the training of a full machine learning model is not the best way to proceed, we will, however, rely on a set of hard-coded rules and heuristics to find matching items.
The 5 categories are the following:
- companies
- addresses
- serial numbers
- item names
- locations

for each of them, we will implement a similar approach with few relevant differences bases on the category.

In [4]:
companies = [
    "Marks and Spencers Ltd",
    "M&S Limited",
    "NVIDIA Ireland",
]

addresses = [
    "SLOUGH SE12 2XY",
    "33 TIMBER YARD, LONDON, L1 8XY",
    "44 CHINA ROAD, KOWLOON, HONG KONG",
]
serials = [
    "XYZ 13423 / ILD",
    "ABC/ICL/20891NC",
]
items = [
    "HARDWOOD TABLE",
    "PLASTIC BOTTLE",
]

locations = [
    "LONDON",
    "HONG KONG",
    "ASIA",
]

### Companies

We will assume that the corporation type of a company is irrelevant for its classification, this will allow the use of fuzzy matching techniques that would fail otherwise.
We also assume that if the location of the company is specified in its name it will be relevant for its classification. E.g. Nvidia Ireland will be a different entity than Nvidia Germany.

The following preprocessing steps are taken:
- substitution of the common ampersand character with the word "and"
- removal of punctuation
- making the strings lowercase
- removal of corporation type acronyms

The removal of the company-type acronyms could lead to some relevant information being lost by the system, it is however the best way to solve this task with this amount of training examples while avoiding conflicts in matching.

The deduplication of the entities is then done by using a fuzzy matcher allowing the matching of acronyms and partial words.

In a more complex system, usage of a trained knowledge-based would be much more effective, however, due to the low volume of the data this manual approach will be more effective for now. For future implementations, the SpaCy entity linking module is very effective given a relevant knowledge base.

An interesting problem is the use of acronyms in documents, this is tricky to deal with due to the possible use of the same acronym for many words. For example, both "Marks and Spencers Ltd" and "Mason and Spears L.T.D." could be mapped to "M&S Limited". In a real-world situation, this problem could probably be ameliorated by using a heuristic based on the context of the document. If a company is mentioned in the document and then its acronym is used then we could be fairly sure that they are a match. At the same time if the acronym never appeared in any of the previous documents would be safe to assume for it to be a new entity. These heuristics should also account for the popularity of some acronyms, AMD ltd is a fairly popular company and a match in a knowledge base could be used to identify it.

In [5]:
# change ampersand
companies = [c.translate(str.maketrans({"&": " and "})) for c in companies]
# make lowercase
companies = [c.lower() for c in companies]
# remove punctuation
companies = [c.translate(str.maketrans("", "", string.punctuation)) for c in companies]
# company type normalization
companies = [
    basename(c, company_terms, prefix=True, middle=False, suffix=True)
    for c in companies
]

In [6]:
print(fuzz.token_set_ratio(companies[0], companies[1]))

60


In [7]:
process.dedupe(companies, threshold=59, scorer=fuzz.token_set_ratio)

dict_keys(['marks and spencers', 'nvidia ireland'])

### Serial numbers

The case of serial numbers is an easier one, they are extremely specific and we can be safe to assume they map the products in a non-continuous way (a small change in the serial number may mean a different product). They are also very sensitive to the position of the characters and typo detection should be avoided.
We can also assume that the punctuation and the spacing in a serial number are only for readability reasons, allowing us to strip it. For these reasons will use a much more strict edit distance to compare serial numbers.

In [8]:
# make lowercase
serials = [c.lower() for c in serials]
# remove punctuation and spaces
serials = [
    c.translate(str.maketrans("", "", string.punctuation + " ")) for c in serials
]

In [9]:
# use stricter Levensthein distance
process.dedupe(serials, threshold=90, scorer=fuzz.ratio)

['xyz13423ild', 'abcicl20891nc']

### Items

The process for items will be similar to the previous ones, the order of the words in the strings should not matter much but the string themselves should. A fuzzy matching ignoring string order should work.

In [10]:
items = [c.lower() for c in items]
items = [
    c.translate(str.maketrans("", "", string.punctuation)) for c in items
]

In [11]:
process.dedupe(items, threshold=80, scorer=fuzz.token_set_ratio)

['hardwood table', 'plastic bottle']

### Addresses 
The normalization of addresses would straightforward by using the google maps API, there is a limited number of unique locations and the Google Maps API has a fully trained disambiguation model that works well.
An alternative way to do this while avoiding the costs of the API requests would be to use the OpenStreetMap data that is freely available and easy to parse locally, at the cost of a higher computational expense.
This approach however brings some edge cases that should be dealt with. Many addresses do not have a unique match if not fully specified and this could lead to bad performance and mismatches. 
A good way to solve this would be to compare the distance of the possible matches to the known location where a company operates in a knowledge base, the closest address would be the most probable match. Also asking for user input for disambiguation would be a good solution.

Due to the cost of the Google Maps API and the need for much more data to be able to parse all types of addresses we will rely on an external library to perform a much simpler address parsing to normalize the order of the elements and then use a fuzzy matcher again.

In [12]:
# make lowercase
addresses = [c.lower() for c in addresses]
# # remove punctuation
addresses = [
    c.translate(str.maketrans(" ", " ", string.punctuation)) for c in addresses
]
# tag the parts of the address and sort them alphabetically to make them have the same order
for a in addresses:
    parsed_address = usaddress.tag(a)
    elements = parsed_address[0]
    elements = OrderedDict(sorted(elements.items(), key=lambda item: item[0]))

    a = " ".join(elements.values())

In [13]:
process.dedupe(addresses, threshold=70, scorer=fuzz.ratio)

['slough se12 2xy',
 '33 timber yard london l1 8xy',
 '44 china road kowloon hong kong']

### Locations
Locations are probably the most difficult items to deal with, many libraries are available but few of them are good at getting a good match. A fuzzy matcher is used again but many improvements could be achieved by having more data to evaluate what is the possible value of the field to manually craft a better parser.

In [14]:
# make lowercase
locations = [c.lower() for c in locations]
# # remove punctuation
locations = [c.translate(str.maketrans("", "", string.punctuation)) for c in locations]

In [15]:
process.dedupe(locations, threshold=70, scorer=fuzz.ratio)

['london', 'hong kong', 'asia']

## Second Task

The second task is very different from the first one, we need to match the already seen entities but we can only access a continuous stream of them and we do not have access to their category beforehand.
The unavailability of the category makes the use of previously devised techniques much harder because we cannot directly apply the pipeline for the right class.

For this reason, a good solution would be to create a classifier able to distinguish the category of the input.
After testing various approaches based on the SpaCy NER module and the GloVe word embeddings we struggled to find a model effective for the low amount of data in this task.

Without access to the class of the input a different pipeline had to be built for this task, the only classification we did is to find if the input is an address as the probability of having conflicts between locations and addresses is very high. By separating the pipeline for addresses and locations we can use a fuzzy matching on the items on the assumption that the content of the various categories is different enough.

In [16]:
companies = [
    "Marks and Spencers Ltd",
    "M&S Limited",
    "NVIDIA Ireland",
    "INTEL LLC",
]

addresses = [
    "SLOUGH SE12 2XY",
    "33 TIMBER YARD, LONDON, L1 8XY",
    "44 CHINA ROAD, KOWLOON, HONG KONG",
]
serials = [
    "XYZ 13423 / ILD",
    "ABC/ICL/20891NC",
    "ICNAO02312",
]
items = [
    "HARDWOOD TABLE",
    "PLASTIC BOTTLE",
    "TOYS",
]

locations = [
    "LONDON",
    "LONDON, ENGLAND",
    "HONG KONG",
    "LONDON, GREAT BRITAIN",
    "ASIA",
]
#%%
stream = [i for c in [companies, serials, locations, items, addresses] for i in c]
stream

['Marks and Spencers Ltd',
 'M&S Limited',
 'NVIDIA Ireland',
 'INTEL LLC',
 'XYZ 13423 / ILD',
 'ABC/ICL/20891NC',
 'ICNAO02312',
 'LONDON',
 'LONDON, ENGLAND',
 'HONG KONG',
 'LONDON, GREAT BRITAIN',
 'ASIA',
 'HARDWOOD TABLE',
 'PLASTIC BOTTLE',
 'TOYS',
 'SLOUGH SE12 2XY',
 '33 TIMBER YARD, LONDON, L1 8XY',
 '44 CHINA ROAD, KOWLOON, HONG KONG']

In [17]:
# testing SpaCy classifier
for item in stream:
    doc = nlp(item)
    ents = [(e.text, e.label_) for e in doc.ents]
    print(ents) 

[('Marks', 'ORG')]
[('M&S Limited', 'ORG')]
[]
[('INTEL', 'ORG')]
[('XYZ 13423 / ILD', 'ORG')]
[('ABC', 'ORG')]
[]
[('LONDON', 'GPE')]
[('LONDON', 'GPE'), ('ENGLAND', 'PERSON')]
[('HONG KONG', 'GPE')]
[('LONDON', 'GPE'), ('GREAT BRITAIN', 'FAC')]
[('ASIA', 'LOC')]
[('HARDWOOD TABLE', 'ORG')]
[]
[]
[('SLOUGH', 'ORG')]
[('33', 'CARDINAL'), ('LONDON', 'GPE'), ('8XY', 'CARDINAL')]
[('44', 'CARDINAL'), ('KOWLOON', 'ORG'), ('HONG KONG', 'GPE')]


In [18]:
class StreamProcessor:
    def __init__(self, threshold=60):
        self.company_terms = prepare_terms()
        self.entities = defaultdict(list)
        #use different thresholds based on the category
        self.thresholds = {"address": 80, "other": 59}

    def process_element(self, item):
        item_class = "other"
        try:
            parsed_address = usaddress.tag(item)
        except usaddress.RepeatedLabelError:
            parsed_address = (None, None)
        # if the item is an address we remove the name of the city to avoid conflicts
        if parsed_address[1] == "Street Address":
            parsed_address[0].pop("PlaceName")
            item = " ".join(parsed_address[0].values())
            
            item_class == "address"
        for entity in self.entities.keys():
            for alias in self.entities[entity]:
                # if the current element matches any of the already known aliases save it as a new alias
                if fuzz.token_set_ratio(self.preprocess(alias), self.preprocess(item))>= self.thresholds[item_class]:
                    self.entities[entity].append(item)
                    return True
        # if not create a new entity using the original string and add itself as a alias
        self.entities[item].append(item)

    def preprocess(self, item):
        # change ampersand
        item = item.translate(str.maketrans({"&": " and "}))
        # make lowercase
        item = item.lower()
        # company type normalization
        item = basename(item, company_terms, prefix=True, middle=False, suffix=True)
        # remove punctuation
        item = item.translate(str.maketrans("", "", string.punctuation))
        return item

In [19]:
processor = StreamProcessor()

for s in stream:
    processor.process_element(s)

In [20]:
processor.entities

defaultdict(list,
            {'Marks and Spencers Ltd': ['Marks and Spencers Ltd',
              'M&S Limited'],
             'NVIDIA Ireland': ['NVIDIA Ireland'],
             'INTEL LLC': ['INTEL LLC'],
             'XYZ 13423 / ILD': ['XYZ 13423 / ILD'],
             'ABC/ICL/20891NC': ['ABC/ICL/20891NC'],
             'ICNAO02312': ['ICNAO02312'],
             'LONDON': ['LONDON', 'LONDON, ENGLAND', 'LONDON, GREAT BRITAIN'],
             'HONG KONG': ['HONG KONG'],
             'ASIA': ['ASIA'],
             'HARDWOOD TABLE': ['HARDWOOD TABLE'],
             'PLASTIC BOTTLE': ['PLASTIC BOTTLE'],
             'TOYS': ['TOYS'],
             'SLOUGH SE12 2XY': ['SLOUGH SE12 2XY'],
             '33 TIMBER YARD L1 8XY': ['33 TIMBER YARD L1 8XY'],
             '44 CHINA ROAD': ['44 CHINA ROAD']})

## Conclusion

All of these models are prototypes only, a much more in-depth study should be done to see the characteristics of the data and their distribution so that more effective parsing techniques can be developed.

The most interesting avenue for this task is the linking to a knowledge-base trained on past data and able to use information about the final use for the information. Given the focus on document parsing of Vector, an interesting avenue would be the building of a knowledge base of the clients and their interactions with location items and other entities. In this way, the matching of already seen entities would be much more accurate and effective.